In [ ]:
#@title Colab setup
import os, sys, subprocess
if "google.colab" in sys.modules:
  cmd = "pip install --upgrade biocircuits bokeh-catplot watermark blackcellmagic intersect"
  process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE, stderr=subprocess.PIPE)
  stdout, stderr = process.communicate()
# ------

# Libraries to solve ODEs
import scipy.integrate
import biocircuits
import numba

# Libraries for mathematical analysis
import numpy as np
from intersect import intersection
from skimage import measure
import sympy as sym
import pylab as pl

# Libraries to visualize results
import bokeh.io
import bokeh.plotting
import bokeh.palettes
from bokeh.models import LinearColorMapper, ColorBar
from bokeh.models import Range1d
from bokeh.io import export_svgs

bokeh.io.output_notebook()

In [ ]:
#@title Auxiliary functions

# Function to do the 2D projection of decision boundaries with normalization
def contourf(p, x, y, z, title=None, palette="Spectral11", normal = True):
    """Make a filled contour plot given x, y, z data given in 2D arrays."""

    # Normalize the z values
    if normal:
      z_min = z.min()
      z_max = z.max()
      z_normalized = (z - z_min) / (z_max - z_min)  # Normalize to range [0, 1]
    else:
      z_normalized = z

    # Add zero padding at the boundaries for better visualization
    N = z_normalized.shape[1]
    z0 = np.c_[z_normalized, np.zeros(N)]
    z0[-1, -1] = 1.  # Ensure the padding contains a value within range

    # Plot the normalized values
    p.image(
        image=[z0],
        x=x.min(),
        y=y.min(),
        dw=(x.max() - x.min()) * (1 + 1 / N),
        dh=x.max() - x.min(),
        palette=palette,
        alpha=0.8,
    )

    # Color mapping based on the normalized z0 values
    color = LinearColorMapper(palette=palette, low=z0.min(), high=z0.max())
    cb = ColorBar(color_mapper=color, location=(0, 0), width=10)
    p.add_layout(cb, 'right')

    return ()

## Sequestration-based neural network: competition for the RNA polymerase (RNAp)


Model of a competitive dimerization binding reaction in which two sigma factors ($S_1, S_2$) compete to bind with RNA polymerase (RNAp), forming active transcription complexes ($C_1, C_2$, respectively). Each sigma factor has a corresponding anti-sigma molecule ($A_1, A_2$), which binds to it to form a biochemically inactive complex. The system is governed by the following equations, obtained from [Moghimianavval et al (2024)](https://pubs.acs.org/doi/full/10.1021/acssynbio.4c00270):

\begin{aligned}
    \frac{dS_1}{dt} &= a_1 - d S_1 - \gamma_1 A_1 S_1 - \gamma_2 S_1 C, \\
    \frac{dA_1}{dt} &= b_1 - d A_1 - \gamma_1 A_1 S_1, \\
    \frac{dS_2}{dt} &= a_2 - d S_2 - \gamma_1 A_2 S_2 - \gamma_2 S_2 C, \\
    \frac{dA_2}{dt} &= b_2 - d A_2 - \gamma_1 A_2 S_2, \\
    \frac{dC_1}{dt} &= \gamma_2 S_1 C - d C_1, \\
    \frac{dC_2}{dt} &= \gamma_2 S_2 C - d C_2
\end{aligned}

and

\begin{aligned}
  C = C_t - C_1 - C_2
\end{aligned}

  Where:
 - $C_t$ : total RNAp [μM]
 - $C$ : free RNAp [μM]

 - $a_1, a_2$ : syntesis rate of sigma factors [μMh$^{-1}$]
 - $b_1, b_2$ : syntesis rate of anti-sigma molecules [μMh$^{-1}$]
 - $γ_1$ : Sigma-antisigma complex formation rates	[μM$^{-1}$h$^{-1}$]
 - $γ_2$ : Sigma-RNApol complex formation rates	[μM$^{-1}$h$^{-1}$]
 - $d$ : degradation rates  [h$^{-1}$]


In [ ]:
#@title Non-Linear decision with competition -- varying RNApol concentration

def Sequestration_rhs(x,t,a1,b1,a2,b2,g1,g2,d,ct):
    S1, A1, S2, A2, C1, C2 = x
    C = ct - C1 - C2
    return np.array(
        [
            a1 - d*S1 - g1*A1*S1 - g2*S1*C,
            b1 - d*A1 - g1*A1*S1,
            a2 - d*S2 -  g2*S2*C,
            b2 - d*A2 - g1*A2*S2,
            g2*S1*C - d*C1,
            g2*S2*C - d*C2,
        ]
    )

g1 = 1000.           # Sigma-antisigma complex formation rates	[1/μMh]
g2 = 10.             # Sigma-RNApol complex formation rates [1/μMh]
d = 1.               # degradation Rates [1/h]
ct = [0.2, 0.6, 1.]  # RNApol total [μMh]

a2 = 0               # Sigma2 syntesis rate [μM/h]
b2 = 0.5             # Antisigma2 syntesis rate [μM/h]


## Parameters for simulation
tN = 100
x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

# Inputs
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)
# Outputs
output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

# Effect of diferent concentrations of RNApol
for k, ctk in enumerate(ct):
    # For each combination of imputs x1, x2, save the final outputs C1 and C2
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            # Node 1 weights
            a1 = x1i
            b1 = x2j
            # Node 2 weights
            a2 = x1i + x2j
            b2 = b2
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, a2, b2,  g1, g2, d, ctk))
            output1[j,i,k] = x.transpose()[4,-1]/ctk # (c1)
            output2[j,i,k] = x.transpose()[5,-1]/ctk # (c2)

# Set up plots
fig_size = (150,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

color = LinearColorMapper(palette = palette, low = z0.min(), high = z0.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
#plots[0].add_layout(cb, 'right')

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # bottom C2

In [ ]:
#@title Linear decision without competition -- varying RNApol concentration

g1 = 1000.           # Sigma-antisigma complex formation rates	[1/μMh]
g2 = 10.             # Sigma-RNApol complex formation rates [1/μMh]
d = 1.               # degradation Rates [1/h]
ct = [0.2, 0.6, 1.]  # RNApol total [μMh]

a2 = 0               # Sigma2 syntesis rate [μM/h]
b2 = 0.5             # Antisigma2 syntesis rate [μM/h]


## Parameters for simulation
tN = 100
x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

# Inputs
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)
# Outputs
output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

# Effect of diferent concentrations of RNApol
for k, ctk in enumerate(ct):
    # For each combination of imputs x1, x2, save the final outputs C1 and C2
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            # Node 1 weights
            a1 = x1i
            b1 = x2j
            # Node 2 weights
            a2 = x1i + x2j
            b2 = b2
            # Node 1 isolated:
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, 0, 0,  g1, g2, d, ctk))
            output1[j,i,k] = x.transpose()[4,-1]/ctk # (c1)
            # Node 2 isolated
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(0, 0, a2, b2,  g1, g2, d, ctk))
            output2[j,i,k] = x.transpose()[5,-1]/ctk # (c2)

# Set up plots
fig_size = (150,115)
#fig_size = (130,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(ct):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # bottom C2

In [ ]:
#@title Non-Linear decision with competition -- varying $\gamma_1$


g1 = [1000., 100., 10.]           # Sigma-antisigma complex formation rates	[1/μMh]
g2 = 10.             # Sigma-RNApol complex formation rates [1/μMh]
d = 1.               # degradation Rates [1/h]
ct = 1/5             # RNApol total [μMh]

a2 = 0               # Sigma2 syntesis rate [μM/h]
b2 = 0.5             # Antisigma2 syntesis rate [μM/h]


## Parameters for simulation
tN = 100
x0 = np.array([0., 0., 0., 0., 0., 0.])
t = np.linspace(0,5,tN)

# Inputs
N = 21
x1 = np.linspace(0,1,N)
x2 = np.linspace(0,1,N)
# Outputs
output1 = np.zeros((N,N,3))
output2 = np.zeros((N,N,3))

# Effect of diferent concentrations of RNApol
for k, g1k in enumerate(g1):
    # For each combination of imputs x1, x2, save the final outputs C1 and C2
    for i, x1i in enumerate(x1):
        for j, x2j in enumerate(x2):
            # Node 1 weights
            a1 = x1i
            b1 = x2j
            # Node 2 weights
            a2 = x1i + x2j
            b2 = b2
            x = scipy.integrate.odeint(Sequestration_rhs, x0, t, args=(a1, b1, a2, b2,  g1k, g2, d, ct))
            output1[j,i,k] = x.transpose()[4,-1]/ct # (c1)
            output2[j,i,k] = x.transpose()[5,-1]/ct # (c2)

# Set up plots
fig_size = (150,115)
x_range = (x1[0],x1[-1])
y_range = (x2[0],x2[-1])

palette = bokeh.palettes.grey(15)[::-1]
palette = bokeh.palettes.Blues9[::-1]

plots = []
for i in range(6):
    plots.append(bokeh.plotting.figure(width=fig_size[0], height=fig_size[1],
                          x_range=x_range, y_range=y_range),)

# A(2w1) B(w1) C(2w2)
# t=1h, 2h, 3h, 5h
for k, itr in enumerate(g1):
    z0 = np.c_[output1[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

    z0 = np.c_[output2[:,:,k], np.zeros(N)]
    z0[-1,-1] = 1.
    plots[k+3].image(
        image=[z0],
        x=x1.min(), y=x2.min(),
        dw=(x1.max() - x1.min())*(1 + 1/N),
        dh=x2.max() - x2.min(),
        palette=palette,
        alpha=0.8,
        )

color = LinearColorMapper(palette = palette, low = z0.min(), high = z0.max())
cb = ColorBar(color_mapper = color, location = (0,0), width=5)
#plots[0].add_layout(cb, 'right')

bokeh.io.show(bokeh.layouts.row(plots[0:3])) # Top C1
bokeh.io.show(bokeh.layouts.row(plots[3:6])) # bottom C2